In [3]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import numpy.random as rnd
from scipy import optimize
import scipy
import random
import math

In [5]:
# Constants 

pi = np.pi
c = 3*10**8
e = 1.6*10**(-19)
me = 9.1*10**(-31)
E_0 = 1
w_0 = 100
t_pulse = 10**(-9)
omega_at = 1
Z = 18 #Arg
pi = np.pi
E_at = 1


In [7]:

class Maxwell:
    
    """Solves maxwell equations with interaction with the plasma"""
    
    def __init__(self, L, N_l, T, EI, l, m, E0, omega0, t_pulse) :
        self.L = L            #interation length
        self.T = T          #time of the simulation
        self.N_l = N_l        #number of spatial steps
        self.dx = L/N_l           #spatial step
        self.dt = c*self.dx       #time step
        N_t = int(T/self.dt)        #number of time steps
        self.N_t = N_t
        self.Eright = np.zeros((N_l, N_t))    #transmitted field
        self.Eleft = np.zeros((N_l, N_t))        #reflected field
        self.J = np.zeros((N_l, N_t))       #transverse current
        self.N0 = np.zeros((N_l, N_t))
        self.N1 = np.zeros((N_l, N_t))
        self.EI = EI              #array of ionization potentials
        self.l = l             #array of azimutal quantum numbers
        self.m = m              #array of magnetic quantum numbers
        self.E0 = E0            #amplitude of initial pulse
        self.omega0 = omega0          #pulsation of initial pulse
        self.t_pulse = t_pulse            #duration of initial pulse
        

    def pulse(self, t):
        '''Returns the boundary condition of the transmitted field which is the initial pulse'''
        return self.E0 * np.sin(self.omega0 * t) * np.exp(-(t/self.t_pulse)**2)
    
    def step(self, nt):
        self.population(nt)
        self.current(nt)
        self.electrical(nt)
    
    def electrical(self, n_t):
        '''Returns Eleft and Eright at time n_t+1'''
        dt = self.dt
        N_l = self.N_l
        self.Eright[0, n_t+1] = self.pulse((n_t+1)*dt)
        self.Eleft[N_l-1, n_t+1] = 0
        for n_l in range(N_l-1):
            J = self.J[n_l, n_t+1] + self.J[n_l+1, n_t+1]
            self.Eright[n_l+1, n_t+1] = self.Eright[n_l, n_t] - np.pi * dt * J
            self.Eleft[n_l, n_t+1] = self.Eleft[n_l+1, n_t] - np.pi * dt * J
            
    def current(self, n_t):
        '''Return the current at time dt(n_t+1/2)'''
        dt = self.dt
        N_l = self.N_l
        alpha = e**2 * dt/m
        for n_l in range(N_l):
            E = self.Eleft[n_l, n_t] + self.Eright[n_l, n_t]
            ne = self.N1[n_l, n_t]
            self.J[n_l, n_t+1] = self.J[n_l, n_t] + alpha * ne * E 
    
    def rate(self, k, n_l, n_t):
        '''Calculates the ionization rate for the level k at time dt*(n_t+1) and position dx*n_l'''
        E = self.Eleft[n_l, n_t] + self.Eright[n_l, n_t]
        lk = self.l[k]       # azimutal quantum number of level k
        mk = self.m[k]       # magnetic quantum number of level k
        mk = np.abs(mk)

        epsilon_k = self.EI[k]   
        fact = (2*lk+1)*math.factorial(lk+mk)/(2**(mk)*math.factorial(mk)*math.factorial(lk-mk))
        n_star = (epsilon_h/epsilon_k)**0.5*Z
        C_nstar = (2*np.ex/n_star)**n_star*(2*pi*n_star)**(-0.5)

        R_st = omega_at*C_nstar**2*epsilon_i/epsilon_h*fact*(2*(epsilon_k/epsilon_h)**1.5*E_at/E)**(2*n_star-m-1)
        R_st = R_st*np.exp(-2/3*(epsilon_k/epsilon_h)**1.5*E_at/E)

        return R_st

    def population(self, n_t):
        '''Calculate the populations density at time dt*(n_t+1)'''
        for n_l in range(N_l):
            R0 = self.rate(0, n_l, n_t) 
            R1 = self.rate(1, n_l, n_t)
            self.N0[nl, n_t+1] = (1 - R0*self.dt)*self.N0[nl, n_t]
            self.N1[nl, n_t+1] = (1 - R1*self.dt)*self.N1[nl, n_t] + R0*self.dt*self.N0[nl, n_t]
